# Agent Orchestration with a Planner

In this workshop, we will use the planner to design a multi-agent **FLEET** designated for the task of generating a cybersecurity report. The following agents will be included:

1. time_keeper
2. cyber_collector
3. db_reader
4. data_analyzer
5. security_evaluator
6. report_writer

In [2]:
import os
import logging
from dotenv import load_dotenv


logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(
    logging.WARNING)

load_dotenv()
print(os.getenv('CHAT_MODEL'))

gpt-4o-mini-2


# List all agents

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(exclude_managed_identity_credential=True), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [4]:
fleet_name = "internet_threat_analysis"
agent_fleet = []
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    if "group" in _agent.metadata.keys() and _agent.metadata["group"] == fleet_name:
        agent_fleet.append({"id": _agent.id, 
                            "name": _agent.name,
                            "description": _agent.description})
        
agent_fleet

[{'id': 'asst_7jQ3Oj0KVGIqfT04FRIE4oPl',
  'name': 'report_writer',
  'description': 'An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.'},
 {'id': 'asst_RJrhrkh39l3q4srGZNEV6VrF',
  'name': 'report_writer',
  'description': 'An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.'},
 {'id': 'asst_zusQwwRqrFxYVtoHnCT85HPb',
  'name': 'security_evaluator',
  'description': 'AI agent that assists users in generating radar charts for cybersecurity metrics.'},
 {'id': 'asst_42dIHjKA22YR02VstBHialdB',
  'name': 'security_evaluator',
  'description': 'AI agent that assists users in generating radar charts for cybersecurity metrics.'},
 {'id': 'asst_QBxhE6ZP6hBy6Ybjxl1Fsv32',
  'name': 'data_analyzer',
  'description': 'An agent that analyzes data usin

In [5]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

deployment_name = os.environ["CHAT_MODEL"]

In [6]:
sys_prompt = """You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[__AGENT_LIST_PLACEHOLDER__]

Make sure to consider the descriptions of each agent when deciding which one to allocate. If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer. Your goal is to provide clear and helpful information to the user, whether through your own knowledge or via the correct agent.

## Constraints
- Do not allocate an agent if you can answer the question directly.
- If there is an agent that can possibly answer the question, allocate that agent.
- If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer.
- If you don't allocate an agent, you must respond with the prefix [__PLANNER__], followed by your response. For example, [__PLANNER__]Hello, I'm here to assist you.;
- If you allocate an agent, you must respond with the prefix [__AGENT__], followed by the agent's name, and then [__TASK__] followed by the request to that agent. For example: [__AGENT__]HR Helpdesk;[__TASK__]Get the company benefit policies.;
- If you think the user's question is fully answered, respond with the prefix [__TERMINATION__] followed by your response. Your response should provide all relevant information and download links matching the user's query, followed by a closing message. Don't just make a summary of the conversation, but provide a clear and concise answer to the user's query including relevant links.
"""

In [7]:
import json

cyber_fleet_str = json.dumps(agent_fleet).replace("{", '{{').replace("}", '}}')
print(sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str))

You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[{{"id": "asst_7jQ3Oj0KVGIqfT04FRIE4oPl", "name": "report_writer", "description": "An advanced AI agent responsible f

In [8]:
addtional_constraint = '''
## Guidelines to generate a report
1. Start with the time keeper to get the current time.
2. Allocate the cyber collector to retrieve latest threat information.
3. Use the metric types from the cyber collector to have the db reader fetch database data.
4. Use the CSV files from the db reader to have the data analyzer perform analysis, including forecasting and anomaly detection.
5. Allocate the security evaluator to create radar chart of evaluation metrics.
6. Start report writer to compile information and generate PDF report.
'''

In [9]:
from user_functions import user_functions
from azure.ai.projects.models import FunctionTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

functions = FunctionTool(functions=user_functions)


def agent_execution(agent_id, task, context):

    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content=f'task: {task} \n\n context: {context}')

    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_run(
        thread_id=thread.id, assistant_id=agent_id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(
                    thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    return messages

In [10]:
def get_assistant_content(agent_messages):
    contents = []
    for message in agent_messages:
        if message['role'] == 'assistant':
            contents.append(message['content'][0]['text']['value'])
    return contents

In [11]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for Contoso. The security evaluation metrics are: Vulnerability Score(8), Detection Rate(7), Response Time(6), Threat Intelligence(9), System Uptime(8)"
    },
]

In [12]:
response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

llm_response = response.choices[0].message.content

messages.append({"role": "assistant", "content": f"{llm_response}"})
while "[__AGENT__]" in llm_response:
    agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
    print(f"Allocated agent: {agent_name}")
    agent_task = llm_response.split("[__TASK__]")[1]
    print(f"Agent task: {agent_task}")
    # find agent id from agent fleet based on agent_name
    agent_id = None
    for agent in agent_fleet:
        if agent["name"] == agent_name:
            agent_id = agent["id"]
            break

    if agent_id:
        agent_messages = agent_execution(agent_id, agent_task, json.dumps(messages[1:]))
        agent_contents = get_assistant_content(agent_messages.data)
        if agent_contents:
            for agent_content in agent_contents:
                messages.append(
                    {"role": "assistant", "content": f"[__AGENT__({agent_name})]{agent_content}"})

        response = aoai_client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=1000
        )
        llm_response = response.choices[0].message.content
        print(llm_response)
        messages.append({"role": "assistant", "content": f"{llm_response}"})
    else:
        break

print(messages)

Allocated agent: time_keeper
Agent task: Get the current time.
Created thread, ID: thread_VCAoEAcsOlHY7gxHX6S9PxQq
Created message, ID: msg_dCfDgKyaQslW3F40FbO9ZeD5
Created run, ID: run_3Nleid8X1UetfaNQfoCkd60C
Current run status: RunStatus.QUEUED
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Executing tool call: {'id': 'call_hpNNGzW8ByFxCTPYnzAsC2xF', 'type': 'function', 'function': {'name': 'fetch_current_datetime', 'arguments': '{"format":null}'}}
Tool outputs: [{'tool_call_id': 'call_hpNNGzW8ByFxCTPYnzAsC2xF', 'output': '{"current_time": "2025-04-09 07:42:18"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Curren

ERROR:xhtml2pdf.files:gaierror: [Errno 11001] getaddrinfo failed while extracting data from NetworkFileUri: 'https://via.placeholder.com/600x400?text=Performance+Metrics+Chart' on attempt 1


Executing tool call: {'id': 'call_3GVyZtx7oWRZbjgcE6nxfxBu', 'type': 'function', 'function': {'name': 'generate_report', 'arguments': '{"markdown_content":"# Cybersecurity Threat and Defense Report\\n\\n**Date:** 2025-04-09 07:42:18  \\n**Client:** Contoso\\n\\n---\\n\\n## Table of Contents\\n1. [Overview](#overview)  \\n2. [Threat Intelligence](#threat-intelligence)  \\n3. [Data Analysis Results](#data-analysis-results)  \\n4. [Defensive Recommendations](#defensive-recommendations)  \\n5. [Compliance Assessment](#compliance-assessment)  \\n6. [Conclusion](#conclusion)  \\n7. [Appendices](#appendices)  \\n\\n---\\n\\n## Overview  \\nThis report provides an evaluation of Contoso\'s cybersecurity posture based on key performance metrics. The objective is to assess the current landscape of cybersecurity threats and propose actionable steps to enhance defenses.\\n\\n## Threat Intelligence  \\nRecent analysis shows a high level of threat intelligence with a score of **9**. This indicates ef

ERROR:xhtml2pdf.files:gaierror: [Errno 11001] getaddrinfo failed while extracting data from NetworkFileUri: 'https://via.placeholder.com/600x400?text=Performance+Metrics+Chart' on attempt 2
ERROR:xhtml2pdf.files:gaierror: [Errno 11001] getaddrinfo failed while extracting data from NetworkFileUri: 'https://via.placeholder.com/600x400?text=Performance+Metrics+Chart' on attempt 3
'<img alt="Performance Metrics" src="https://via.placeholder.com/600x400?text=Performance+Metrics+Chart"/>'


Tool outputs: [{'tool_call_id': 'call_3GVyZtx7oWRZbjgcE6nxfxBu', 'output': './reports/report_20250409_074247.pdf'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


INFO:openai._base_client:Retrying request to /chat/completions in 0.483320 seconds


[__TERMINATION__]The cybersecurity report for Contoso has been generated successfully and is available for download. You can access it using the following link: [Download Cybersecurity Threat and Defense Report](sandbox:/reports/report_20250409_074247.pdf). If you have any further questions or need additional assistance, feel free to ask!
[{'role': 'system', 'content': 'You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.\n\nDirect Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.\n\nAgent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with th